In [1]:
import random
import math
from functools import lru_cache

In [2]:
import metaheuristics as mh

In [3]:
class Michalewicz(mh.abstract.Problem):
    
    def __init__(self, d, m, *args, **kwargs):
        self.d = d
        self.m = m
        super().__init__(*args, **kwargs)
    
    def config(self):
        pass

    @property
    def random_solution(self):
        return Solution(self)

    def solution(self, *args, **kwargs):
        return Solution(self, *args, **kwargs)
    
    def reset(self):
        pass
    
    @property
    def difficulty(self):
        return self.d
    
    def __hash__(self):
        return self.m

In [4]:
class Solution(mh.utils.BaseSolutionMixin, mh.abstract.Solution):
    
    def __init__(self, problem, sequence=None):
        super().__init__(problem, sequence=sequence)

    @property
    def sequence(self):
        if not self.__sequence:
            self.__sequence = [random.uniform(0, math.pi) for i in range(self.problem.d)]
        return self.__sequence

    @sequence.setter
    def sequence(self, value):
        self.__sequence = value
        self.__cost = None
        
    @property
    def is_complete(self):
        return not self.is_partial

    @property
    def is_partial(self):
        return False
    
    def _correct(self):
        return True
    
    @property
    def correct(self):
        return self._correct()
    
    @lru_cache(maxsize=2 ** 20)
    def _cost(self):
        return -sum(
            math.sin(x) * (math.sin(((i+1) * x**2)/math.pi))**(2*self.problem.m)
            for i, x in enumerate(self.sequence))
    
    @property
    def cost(self):
        return self._cost()

In [5]:
problem = Michalewicz(d=5, m=5)

In [6]:
ga = mh.algorithms.Genetic(
    problem,
    pop_size=50,
    n_iters=5000,
    crossover=mh.utils.continuous_order_crossover,
    prob_mutation=0.1,
    search_operator=mh.utils.tweak(2, 5, 0, .5)
)
ga()
ga.best_solution

[2.1339442931203787, 1.5681958937023572, 1.3572259647215543, 1.1585242928077155, 2.218042541386396] - -4.23

In [7]:
res = []

for _ in range(10):
    ga = mh.algorithms.Genetic(
        problem,
        pop_size=50,
        n_iters=10_000,
        crossover=mh.utils.continuous_order_crossover,
        prob_mutation=0.1,
        search_operator=mh.utils.tweak(2, 5, 0, .5)
    )
    ga()
    res.append(ga.best_solution.cost)

In [8]:
res

[-4.090002879278165,
 -3.999122377580548,
 -4.004419451626054,
 -4.3093813418725695,
 -3.793699871530604,
 -4.388790476947727,
 -4.418710550944052,
 -4.198773965978493,
 -4.127912680043396,
 -4.492311433852174]